<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/StockAndNewsAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

!pip install pandas-datareader

In [0]:
!pip install -U -q PyDrive

In [0]:
import urllib
import json
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime, date


<h2>Authenticate User </h2>

In [0]:
from google.colab import auth
auth.authenticate_user()


<h3> Loading Nasdaq and Nyse shares </h3>

In [0]:
!gsutil cp gs://datascience-bucket-mm/nyse-companylist.csv /tmp/nyse.csv
!gsutil cp gs://datascience-bucket-mm/nasdaq-companylist.csv /tmp/nyse.csv  
  
# Print the result to make sure the transfer worked.
#!cat /tmp/nyse.csv

Copying gs://datascience-bucket-mm/nyse-companylist.csv...
/ [1 files][392.3 KiB/392.3 KiB]                                                
Operation completed over 1 objects/392.3 KiB.                                    
Copying gs://datascience-bucket-mm/nasdaq-companylist.csv...
/ [1 files][462.5 KiB/462.5 KiB]                                                
Operation completed over 1 objects/462.5 KiB.                                    



<h3>Loading Credentials</h3>

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

def get_iexapi_keys():
  with open('gdrive/My Drive/passwords/iexapi.keys') as f:
    return f.readlines()[0]

def get_nlp_service_keys():
  with open('gdrive/My Drive/passwords/nlp.keys') as f:
    return f.readlines()[0]



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<h3>IEX API CALLS </h3>


In [0]:
import requests

token = get_iexapi_keys()

def get_statistics(ticker):
  base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={token}&format=csv&filter=companyName,symbol,beta,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(token=token,symbol=ticker)
  df = pd.read_csv(base_url)
  df['Symbol'] = ticker
  return df

def get_historical_data(ticker, start, end):
  df = get_statistics(ticker)
  return df


def get_all_stocks():
  all_symbols_data = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token={token}'.format(token=token)).json()
  return [d['symbol'] for d in all_symbols_data if d['isEnabled']and d['type'].lower() == 'cs']

def get_all_us_stocks(security_type='cs'):
  nyse_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nys/symbols?token={token}'.format(token=token)).json()
  #nas_symbols = requests.get('https://cloud.iexapis.com/stable/ref-data/exchange/nas/symbols?token={token}'.format(token=token)).json()
  return [d['symbol'] for d in nyse_symbols  if d['type'].lower() == security_type]

def get_all_etfs():
  stocks= get_all_us_stocks()
  return [d['symbol'] for d in stocks if d['type'].lower() == 'et']

def get_all_stocks_data():
  good_ones = get_all_etfs()
  return map(lambda symbol: (symbol, get_historical_value(symbol)), good_ones)


def get_all_exchanges():
  return requests.get('https://cloud.iexapis.com/stable/ref-data/market/us/exchanges?token={token}'.format(token=token)).json()

def get_latest_price(symbol):
  base_url = "https://cloud.iexapis.com/stable/stock/{ticker}/quote?token={token}&format=csv&filter=symbol,close".format(token=token,ticker=symbol)
  import requests
  return pd.read_csv(base_url)

def get_quote(symbol):
  try:
    historical_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/quote/latestPrice?token={token}'.format(token=token,symbol=symbol)
    return requests.get(historical_url).json()
  except:
    return -1

def get_news(symbol, num_of_news):
  news_url = 'https://cloud.iexapis.com/stable//stock/{symbol}/news/last/{last}?token={token}'.format(symbol=symbol, last=num_of_news,
                                                                                                      token=token)
  return requests.get(news_url).json()


<h3> Yahoo API Calls </h3>

In [0]:
from scipy.stats import pearsonr
import requests

def get_latest_price_yahoo(symbol, as_of_date):
  try:
    return dr.get_data_yahoo(symbol, as_of_date, as_of_date)[['Close']]
  except Exception as e :
    return pd.DataFrame(columns=['Close'])[['Close']]

def get_historical_value(symbol):
  try: 
    data = dr.get_data_yahoo(symbol, date(2018,1,1), date(2019,9,19))[['Adj Close']]
    df =  data.rename(columns={'Adj Close' : symbol})
    return df
  except Exception as e :
    return pd.DataFrame(columns=[symbol])   
  

<h3> Getting Sentiment Analysis from Google </h3>

In [0]:
def get_sentiment_from_google(content):
  from google.cloud import language
  from google.cloud.language import enums
  from google.cloud.language import types

  client = language.LanguageServiceClient()
  document = types.Document(
      content=clean_text,
      type=enums.Document.Type.PLAIN_TEXT)

  # Detects the sentiment of the text
  sentiment = client.analyze_sentiment(document=document).document_sentiment

def test_language_api(content):
  import httplib2
  import sys
  from googleapiclient import discovery
  from googleapiclient.errors import HttpError

  discovery_url = 'https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'

  service = discovery.build(
      'language', 'v1',
      http=httplib2.Http(),
      discoveryServiceUrl=discovery_url,
      developerKey=get_nlp_service_keys(),
  )
  service_request = service.documents().annotateText(
      body={
          'document': {
              'type': 'PLAIN_TEXT',
              'content': content,
          },
          'features': {
              'extract_syntax': True,
              'extractEntities': True,
              'extractDocumentSentiment': True,
          },
          'encodingType': 'UTF16' if sys.maxunicode == 65535 else 'UTF32',
      })
  try:
      response = service_request.execute()
  except HttpError as e:
      response = {'error': e}
  return response



<h3> Reading source data and computing performance </h3>

In [0]:
def get_date_ranges():
  end_date = date.today()
  start_date = end_date - BDay(60)
  return start_date, end_date

def test():
  start,end = get_date_ranges()
  print('start:{}, end:{}'.format(start_date, end_date))
  print(get_historical_data('AMZN', start_date, end_date))
  

  
def get_nyse_df():
  return pd.read_csv('/tmp/nyse.csv', header=0)[['Symbol', 'Name', 'Sector', 'industry']]

def compute_performance(start_dt, end_dt, ticker):
  try:
    import time
    historical_df =  get_historical_data(ticker, start_dt, end_dt)
    latest_df = get_latest_price(ticker)
    merged = pd.merge(historical_df, latest_df, how='inner' , left_on="Symbol", right_on="symbol").drop('symbol', axis=1)
    return merged
  except Exception as e:
    print('Exception:{}'.format(str(e)))
    print('Unable to find data for {}:{}'.format(ticker,str(e)))
    
def find_best_performing(start_dt, end_dt):
  print('Finding Best Performing Stocks between:{}-{}'.format(start_dt, end_dt))
  nyse_df = get_nyse_df()
  symbols = nyse_df['Symbol'].values.tolist()
  print('Now we have to source data for:{}'.format(len(symbols)))
  dfs = (compute_performance(start_dt, end_dt, symbol) for symbol in symbols)
  filtered = (df for df in dfs if df is not None)
  all_data = pd.concat(filtered)
  return pd.merge(nyse_df, all_data, how='inner', on='Symbol' )[['Symbol', 'Name', 'Sector', 'industry', 'companyName','close', 
       'month1ChangePercent','month3ChangePercent', 'month6ChangePercent',  'day200MovingAvg', 'day50MovingAvg']]

  

start_dt, end_dt = get_date_ranges()
perf_df = find_best_performing(start_dt, end_dt)
# Sorting 
perf_df.sort_values(by=['month1ChangePercent'], inplace=True, ascending=False)



<h3> Group by sector, to find best performers </h3>

In [0]:
res = perf_df[['industry', 'month1ChangePercent','month3ChangePercent', ]].groupby(['industry']).mean().sort_values(by=['month1ChangePercent','month1ChangePercent'], ascending=False)
res.head(20)

In [0]:
# Panda test
base_url = 'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token=sk_98e397d4bee940488e1f48e9b419508f&format=csv&filter=companyName,symbol,day50MovingAvg,day200MovingAvg,month6ChangePercent,month3ChangePercent,month1ChangePercent'.format(symbol='AMZN')
df = pd.read_csv(base_url)
df['symbol'] = 'AMZN'
df.columns
  

<p> Testing all stocks in portfolio </p>

In [0]:
portfolio_shares = ['ADAC', 'AMBS', 'AMZN', 'AZFL', 'ARSC', 'AAPL', 'APTY',
                    'BTCS', 'BRK-B', 'CRNT', 'CRLBF', 'XOM', 'HAON', 'AGEEF',
                    'HMNY', 'JNJ', 'LEMIF', 'NXTTF', 'NVCN', 'RNVA', 'TORC',
                    'RTRX', 'VALE', 'VZ', 'DGP', 'RUSL', 'REMX', 'TVIX' ]

all_shares = get_all_stocks()
res = map(lambda symbol:(symbol, symbol in all_shares), shares)
invalid = [tpl for tpl in res if not tpl[1]]


JSONDecodeError: ignored

<h3> Performance Functions </h3>

In [0]:
def calculate_correlation_all(vix, all_stocks):
  all_df = [vix]
  res= [vals for _, vals in all_stocks if vals.shape[0] == vix_vals.shape[0]]
  res.append(vix)
  all_data = pd.concat(res, axis=1)
  return all_data.corr('pearson')

def calculate_portfolio_correlation(all_stocks):
  res= [vals for _, vals in all_stocks if vals.shape[0] > 2]
  all_data = pd.concat(res, axis=1)
  return all_data.corr('pearson')


def calculate_correlation(vix, all_stocks):
  result = []
  best = 0
  for symbol, vals in all_stocks:
    if vals.shape[0] == vix.shape[0]:
      concats  = pd.concat([vix, vals], axis = 1)
      corr_matrix = concats.corr(method='pearson')
      corr_with_vix = corr_matrix.loc['^VIX'][1]
      if corr_with_vix > 0 and corr_with_vix > best:
        print('New Corr with {}:{}'.format(symbol, corr_with_vix))
        best = corr_with_vix
  return best

def _get_most_correlated(result_df):
  df = result_df[['^VIX']]
  bad_df = df.index.isin(['^VIX'])
  return df[~bad_df]
  


In [0]:
#vix_vals = get_historical_value('^VIX')

all_stocks_data = map(lambda symbol: (symbol, get_historical_value(symbol)), portfolio_shares)
best = calculate_portfolio_correlation(all_stocks_data)      


In [0]:
res = _get_most_correlated(best)
sorted_df = res.sort_values('^VIX', ascending=False)
sorted_df.head(10)


In [0]:
idx = sorted_df.iloc[0]

print(idx.values.tolist()[0])
print(sorted_df.index[0])

0.07980534120952935
AAME


In [0]:
sorted_df.index


In [0]:
corr_matrix = all_df.corr(method='pearson')

In [0]:
corr_matrix

In [0]:
from datetime import date
all_stocks = get_all_us_stocks()
print('We got to find:{}'.format(len(all_stocks)))


We got to find:1818


In [0]:
prices = map(lambda s: (s, get_quote(s)), all_stocks[0:2000])
filtered = [tpl for tpl in prices if tpl[1]  > 800]

<h3> REtrieving News </h3>

In [0]:
all_news = get_news('AMZN', 200)
one_news = all_news[2]['summary']

In [13]:
test_language_api(one_news)['documentSentiment']['score']

-0.8

In [0]:
explicit_app_engine('datascience-projects')

OSError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
